In [26]:
import numpy as np
import pandas as pd
import datetime
import csv
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
from keras.models import load_model

from sklearn.metrics import classification_report


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [38]:
# fix random seed:
np.random.seed(7)

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(len(phd))
tf.config.experimental.set_memory_growth(physical_devices[0] , True)

1


#**Data Preparation**

In [ ]:
#Preparing DataSet files paths
#Pathes for Yellow part of the Dataset:
/*
path = []
for m in range(12):
  if(m<9):
    path.append("/content/drive/MyDrive/NYC_TLC_Dataset/Yellow/yellow_tripdata_2018-0" + str(m+1) + ".csv")
  else:
    path.append("/content/drive/MyDrive/NYC_TLC_Dataset/Yellow/yellow_tripdata_2018-" + str(m+1) + ".csv")

print(path)

#Pathes for Green part of the Dataset:
pathG = []
for m in range(12):
  if(m<9):
    pathG.append("/content/drive/MyDrive/NYC_TLC_Dataset/Green/green_tripdata_2018-0" + str(m+1) + ".csv")
  else:
    pathG.append("/content/drive/MyDrive/NYC_TLC_Dataset/Green/green_tripdata_2018-" + str(m+1) + ".csv")

print(pathG)
*/

['/content/drive/MyDrive/NYC_TLC_Dataset/Yellow/yellow_tripdata_2018-01.csv', '/content/drive/MyDrive/NYC_TLC_Dataset/Yellow/yellow_tripdata_2018-02.csv', '/content/drive/MyDrive/NYC_TLC_Dataset/Yellow/yellow_tripdata_2018-03.csv', '/content/drive/MyDrive/NYC_TLC_Dataset/Yellow/yellow_tripdata_2018-04.csv', '/content/drive/MyDrive/NYC_TLC_Dataset/Yellow/yellow_tripdata_2018-05.csv', '/content/drive/MyDrive/NYC_TLC_Dataset/Yellow/yellow_tripdata_2018-06.csv', '/content/drive/MyDrive/NYC_TLC_Dataset/Yellow/yellow_tripdata_2018-07.csv', '/content/drive/MyDrive/NYC_TLC_Dataset/Yellow/yellow_tripdata_2018-08.csv', '/content/drive/MyDrive/NYC_TLC_Dataset/Yellow/yellow_tripdata_2018-09.csv', '/content/drive/MyDrive/NYC_TLC_Dataset/Yellow/yellow_tripdata_2018-10.csv', '/content/drive/MyDrive/NYC_TLC_Dataset/Yellow/yellow_tripdata_2018-11.csv', '/content/drive/MyDrive/NYC_TLC_Dataset/Yellow/yellow_tripdata_2018-12.csv']
['/content/drive/MyDrive/NYC_TLC_Dataset/Green/green_tripdata_2018-01.csv',

In [ ]:
#Reading all the 12 Yellow files:
/*
files = []
for i in range(12):
  f = pd.read_csv(path[i] , usecols=[1 , 7])
  f.columns = ['PU_datetime', 'PU_location']
  files.append(f) 
  
print(len(files))
*/

12


In [ ]:
#Reading all the 12 Green files:
/*
filesG = []
for i in range(12):
  f = pd.read_csv(pathG[i] , usecols=[1 , 5])
  f.columns = ['PU_datetime', 'PU_location']
  filesG.append(f) 
  
print(len(filesG))
*/

12


In [ ]:
#Fractional Sampling:
/*
y = 0
g = 0
samp = []
for i in range(12):
  samp.append(files[i].sample(frac=.000001, random_state=42))
  y += len(samp[i])

for i in range(12):
  samp.append(filesG[i].sample(frac=.00001 , random_state=42))
  g += len(samp[12+i])

print("Number of Yellow samples: " + str(y))
print("Number of Green samples: " + str(g))
print("Total: " + str(y+g))

df = pd.concat(samp)
ds = df.values
print(df)
*/

Number of Yellow samples: 102
Number of Green samples: 89
Total: 191
                 PU_datetime  PU_location
3173938  2018-01-13 09:46:33          229
688471   2018-01-03 19:06:43          144
1822513  2018-01-08 17:51:08          264
4466690  2018-01-17 18:22:23          239
5042422  2018-01-19 14:18:37          236
...                      ...          ...
391213   2018-12-17 17:48:09           25
656838   2018-12-30 16:08:13           41
304967   2018-12-14 06:02:55          145
64509    2018-12-03 21:28:16           74
567382   2018-12-25 20:13:42          181

[191 rows x 2 columns]


In [ ]:
#Save Samples as CSV files (5 samples of different sizes, sample1 is the largest)
/*
df.to_csv("/content/drive/MyDrive/DS_Samples/sample5.csv")
*/

In [ ]:
#Load sample:
sample_number = 5   #1->5 , Largest->Smallest Samples
df = pd.read_csv("/content/drive/MyDrive/DS_Samples/sample" + str(sample_number) + ".csv")
len(df)

191

In [ ]:
# For each time slot t, we will define X=[ n1 , n2 , ...] where each ni corresponds to a zi (for each zone at a time t we have one value for n)
# And we will train the model to predict Y=[ n1 , n2 , ...] at time t+1

#Fixed Number of Zones:
number_of_zones = 265

#Define time slots:
m = 43200  #m is the size of a time slot in minutes
t0 = datetime.datetime(2018,1,1,0,0,0)
number_of_timeSlots = int( 365 * 24 * 60 / m )   
#For each value of i we have time slot t[i]
t = []                                      
for i in range(number_of_timeSlots + 1):  
    ts = datetime.timedelta(minutes=m*i)
    t.append(t0 + ts)

    if(i<5):   #Print 5 instances
      print(t[i])

print("... \nNumber of time slots is: " + str(len(t)))



2018-01-01 00:00:00
2018-01-31 00:00:00
2018-03-02 00:00:00
2018-04-01 00:00:00
2018-05-01 00:00:00
2018-05-31 00:00:00
2018-06-30 00:00:00
2018-07-30 00:00:00
2018-08-29 00:00:00
2018-09-28 00:00:00
2018-10-28 00:00:00
2018-11-27 00:00:00
2018-12-27 00:00:00
 
Number of time slots is: 13


In [ ]:
#Create X and Y:
X = []
Y = []
df["PU_datetime"] = pd.to_datetime(df["PU_datetime"]) #Change type from series to datetime to compare with t[i]'s
#For each value of i we have time slot t[i]
#Each j corresponds to zone number j+1   (Zones numbers starts from 1 till 265)
for i in range(number_of_timeSlots):  
    for j in range(1 , number_of_zones+1):
      
      a = df[ (t[i] <= df["PU_datetime"]) & (df["PU_datetime"] < t[i+1]) & (df["PU_location"] == j)]
      
      if(i == number_of_timeSlots - 1): #At the last time slot we can't find tn+1 anymore, because this will be in 2019.
        Y.append(len(a))                #Then I decided to ignore tn in X and add it to Y only, so that now the last instance in X is at time tn-1 which corresponds to Y at time tn
        continue
        #We can add: if(len(a) != 0 ): append and so on, else leave it to save space... But how will the model understand the "No trip" concept without seeing a zero!?!
      X.append([t[i] , j , len(a)])       #X = [  ... , [ti , zj , nij] , ...]

      if(i == 0):                         #If this is the first interation (i=0) then we only need the value of X. 
        continue                          #We only need Y starting from i=1. So that each entry Y[k] corresponds for prediction of entry X[k]
      Y.append(len(a))                    #Y = [ ... , n(i+1)j , ...]

print(number_of_timeSlots * number_of_zones)
print("Length of X: " + str(len(X)))
print("Length of Y: " + str(len(Y)))

3180
Length of X: 2915
Length of Y: 2915


In [ ]:
number_of_timeSlots * number_of_zones

4642800

In [ ]:
4642800 -  4642535

265

In [ ]:
#Print X instances that have trips:
for i in range(len(X)):
  if(X[i][2] != 0):
    print(str(i) + " , " + str(X[i]))
    if(i == 5):
      break

In [ ]:
#Print Y instances that have trips:
for i in range(len(Y)):
  if(Y[i] != 0):
    print(Y[i])

In [ ]:
24639 - 24374

265

In [ ]:
#Save List X and Y -> CSV File (To Drive):
X_fields = ['TimeSlot', 'Zone' , 'Number_of_trips'] 
Y_fields = ['Number_of_trips']

X_file_to_save_name = "X_Sample5m30" 
Y_file_to_save_name = "Y_Sample5m30"
  
with open("/content/drive/MyDrive/XY_Samples/" + X_file_to_save_name + ".csv", 'w') as fx:
    write = csv.writer(fx)      
    write.writerow(X_fields)
    write.writerows(X)

print("X Saved!")

with open("/content/drive/MyDrive/XY_Samples/" + Y_file_to_save_name + ".csv", 'w') as fy:
    write = csv.writer(fy)      
    write.writerow(Y_fields)
    write.writerows(map(lambda y: [y], Y))

print("Y Saved!")

X Saved!
Y Saved!


#Model Design:

In [27]:
#Load X and Y CSV Files -> List (From Drive):

# sampleNumber = 5
# timeSlotSize = 30

X_file_to_load_name = "X_Sample5monthly" 
Y_file_to_load_name = "Y_Sample5monthly"

X_df = pd.read_csv("/content/drive/MyDrive/XY_Samples/" + X_file_to_load_name +  ".csv")
X_df["TimeSlot"] = pd.to_datetime(X_df["TimeSlot"]) #Convert String -> Datetime
X = X_df.values.tolist() #Dataframe -> List
print(len(X))

Y_df = pd.read_csv("/content/drive/MyDrive/XY_Samples/" + Y_file_to_load_name +  ".csv")
Y = Y_df.values.tolist()
print(len(Y))

2915
2915


In [28]:
#Transforming X and Y into numpy array to be able to fit them to the keras model (we can't fit lists)
X = np.array(X)
Y = np.array(Y)

In [29]:
#Before:
#datetime to integer timestamp
for i in range(len(X)):
  X[i][0] = int(datetime.datetime.timestamp(X[i][0]))

In [ ]:
# #After:
# #integer timestamp to datetime
# for i in range(len(X)):
#   X[i][0] = datetime.datetime.fromtimestamp(X[i][0])

In [ ]:
# #LSTM input_shape must be 3D (samples, time steps , fetures) or  (batch_size, time_steps, seq_len)
# X = X.reshape(1, X.shape[0], 3)
# Y = Y.reshape(1, Y.shape[0], 1)
# print(X.shape)
# print(Y.shape)

(1, 2915, 3)
(1, 2915, 1)


In [30]:
#Train - Test - Split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=0)

print("Train samples: " + str(len(X_train)))
print("Test samples: " + str(len(X_test)))

Train samples: 2332
Test samples: 583


In [31]:
X_train = X_train.reshape(1, X_train.shape[0], 3)
Y_train = Y_train.reshape(1, Y_train.shape[0], 1)
X_test = X_test.reshape(1, X_test.shape[0], 3)
Y_test = Y_test.reshape(1, Y_test.shape[0], 1)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(1, 2332, 3)
(1, 2332, 1)
(1, 583, 3)
(1, 583, 1)


In [ ]:
# #from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler(feature_range = (0,1))
# X_Scaled = scaler.fit_transform(X.reshape(-1,1))

In [ ]:
# Y_Scaled = scaler.fit_transform(Y.reshape(-1,1))

In [ ]:
# #Print X instances that have trips:
# for i in range(len(X_Scaled)):
#   if(X_Scaled[i][2] != 0):
#     print(str(i) + " , " + str(X[i]))
#     if(i == 5):
#       break

In [ ]:
# #Print Y instances that have trips:
# for i in range(len(Y)):
#   if(Y[i] != 0):
#     print(Y[i])

In [35]:
model = Sequential()
model.add(LSTM(265 , input_shape = (X_train.shape[1] , 3) , return_sequences=True))  
model.add(Dropout(0.2)) #To avoid overfitting (Drop 20% of the data)

model.add(LSTM(265 , return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(265 , return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(265))  #return_sequences is False by default
model.add(Dropout(0.2))

model.add(Dense(units = 1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 2332, 265)         285140    
_________________________________________________________________
dropout_1 (Dropout)          (None, 2332, 265)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 2332, 265)         562860    
_________________________________________________________________
dropout_2 (Dropout)          (None, 2332, 265)         0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 2332, 265)         562860    
_________________________________________________________________
dropout_3 (Dropout)          (None, 2332, 265)         0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 265)              

In [ ]:
# #2*24*7 * 265 = 336*265 , one week all zones (case of 30 minutes time slots)
# #1*265 = 265 (case of 1 month time slots)
# model = Sequential()
# model.add(LSTM(265 , input_shape = (265 , 3) , activation='relu' , return_sequences=True))  
# model.add(Dropout(0.2))

# model.add(LSTM(128 , activation='relu'))
# model.add(Dropout(0.2))

# model.add(Dense(32 , activation='relu'))
# model.add(Dropout(0.2))

# model.add(Dense(10 , activation='softmax'))
# model.summary()

In [ ]:
# model = Sequential()
# model.add(LSTM(128 , input_shape=(X_train.shape[1:]) , activation='relu' , return_sequences=True))
# model.add(Dropout(0.2))

# model.add(LSTM(128 , activation='relu'))
# model.add(Dropout(0.2))

# model.add(Dense(32 , activation='relu'))
# model.add(Dropout(0.2))

# model.add(Dense(10 , activation='softmax'))
# model.summary()

#Training:

In [16]:
X_train = np.asarray(X_train).astype('int')
Y_train = np.asarray(Y_train).astype('int')
X_test = np.asarray(X_test).astype('int')
Y_test = np.asarray(Y_test).astype('int')

opt = tf.keras.optimizers.Adam(learning_rate=0.001 , decay=0.000001)
model.compile(loss='mse' , optimizer=opt , metrics = ['accuracy'])
history = model.fit(X_train , Y_train , epochs=10, batch_size = 10, shuffle=True, validation_data=(X_test , Y_test))

Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 1980985902127691513987072.0000 - accuracy: 4.2882e-04 - val_loss: nan - val_accuracy: 0.9554
Epoch 2/10
1/1 [==============================] - 2s 2s/step - loss: nan - accuracy: 0.9434 - val_loss: nan - val_accuracy: 0.9554
Epoch 3/10
1/1 [==============================] - 2s 2s/step - loss: nan - accuracy: 0.9434 - val_loss: nan - val_accuracy: 0.9554
Epoch 4/10
1/1 [==============================] - 2s 2s/step - loss: nan - accuracy: 0.9434 - val_loss: nan - val_accuracy: 0.9554
Epoch 5/10
1/1 [==============================] - 2s 2s/step - loss: nan - accuracy: 0.9434 - val_loss: nan - val_accuracy: 0.9554
Epoch 6/10
1/1 [==============================] - 2s 2s/step - loss: nan - accuracy: 0.9434 - val_loss: nan - val_accuracy: 0.9554
Epoch 7/10
1/1 [==============================] - 2s 2s/step - loss: nan - accuracy: 0.9434 - val_loss: nan - val_accuracy: 0.9554
Epoch 8/10
1/1 [==============================] - 2s

In [37]:
model.get_weights()

[array([[ 0.06232844, -0.02953425,  0.00104604, ...,  0.06917667,
         -0.01752673,  0.02144372],
        [ 0.06741759, -0.02556932,  0.07469261, ...,  0.03084643,
         -0.05912355,  0.05797212],
        [ 0.04930213,  0.030977  ,  0.0502459 , ..., -0.02291288,
          0.01411908, -0.00923607]], dtype=float32),
 array([[-0.00842428, -0.01083936,  0.01340715, ..., -0.01725402,
          0.02310201, -0.00881189],
        [ 0.00255297, -0.00966644,  0.00470481, ..., -0.01024144,
         -0.01192132, -0.0180763 ],
        [ 0.01996517,  0.00919777, -0.01450181, ...,  0.05405999,
          0.0099665 ,  0.00402816],
        ...,
        [ 0.07117599, -0.04026072,  0.04068448, ..., -0.00350265,
         -0.02449436,  0.05995306],
        [ 0.04946085,  0.01118303, -0.01256744, ..., -0.00744581,
         -0.0144234 ,  0.00852701],
        [ 0.00298745,  0.0395629 ,  0.00100014, ..., -0.01380257,
         -0.05685732,  0.00586968]], dtype=float32),
 array([0., 0., 0., ..., 0., 0., 0.

In [21]:
#Save Trained Model to Drive:
#Saving will save the same exact model with all the weights and everything (The trained model itself)
saved_model_name = "modelX"
model.save('/content/drive/MyDrive/Models/' + saved_model_name)

model.save_weights("/content/drive/MyDrive/Models/w")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Models/modelX/assets


In [25]:
#Load Model from Drive:
loaded_model_name = "modelX"
model = load_model('/content/drive/MyDrive/Models/' + loaded_model_name)

model.load_weights("/content/drive/MyDrive/Models/w")

#Checks:
model.summary()
model.get_weights()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 265, 265)          285140    
_________________________________________________________________
dropout (Dropout)            (None, 265, 265)          0         
Total params: 285,140
Trainable params: 285,140
Non-trainable params: 0
_________________________________________________________________


[array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]], dtype=float32),
 array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]], dtype=float32),
 array([nan, nan, nan, ..., nan, nan, nan], dtype=float32)]

In [ ]:
#Note: We can also save the model as JSON String,
#And then save the json string in a file on the drive

#Save:
json_string = model.to_json()

#Load:
from tensorflow.keras.models import model_from_json
m7_Loaded_Model = model_from_json(json_string)

#check: m7_Loaded_Model.summary()

#Testing:

In [ ]:
# y_pred = model.predict(X_test)
# y_pred = np.argmax(y_pred, axis = 1)
# y_test = np.argmax(Y_test, axis = 1)
# print(classification_report(y_test, y_pred))